In [ ]:
from annoy import AnnoyIndex

In [ ]:
import sqlite3
import random
import itertools
import subprocess
import os
import shlex
import time
import pickle
import joblib
from multiprocessing import Pool

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
from matplotlib.colors import Normalize

In [ ]:
from PIL import Image, ImageDraw, ImageFont

In [ ]:
from imageio import imread

In [ ]:
from skimage.transform import resize
from sklearn.decomposition import PCA, IncrementalPCA

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
# import tensorflow_datasets as tfds

In [ ]:
print(tf.__version__)

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import decode_predictions, preprocess_input
from tensorflow.keras.models import Model

# for tf 2
# from keras.applications.imagenet_utils import decode_predictions, preprocess_input

In [ ]:
# this seems to help with some GPU memory issues

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

#### Set hyperparameters

In [ ]:
# mode = "vgg"
mode = "raw"
# mode = "dc"
# mode = "ternary"

In [ ]:
# dataset_size = 2000 # testing
# dataset_size = 120000
# dataset_size = 600000 # for saving IPCA files from 5% of data
dataset_size = 1200000 # for testing larger ann files
batch_size = 2000
num_images = dataset_size

In [ ]:
# ann_filename = "test_5k_pca.ann"
# ann_filename = "120k_ipca.ann"
# ann_filename = "120k_vgg_ipca.ann"

# ann_filename = "2k_pca.ann"
# ann_filename = "2k_ipca.ann"
# ann_filename = "2k_raw.ann"
# ann_filename = "2k_vgg_ipca.ann"
# ann_filename = "2k_ternary.ann"

# ann_filename = "600k_vgg_ipca.ann"
# ann_filename = "600k_raw.ann"
# ann_filename = "600k_ternary.ann"

# ann_filename = f'{dataset_size / 1000000:02.1f}m_{mode}.ann'
ann_filename = f'{dataset_size / 1000:01.0f}k_{mode}.ann'
print(ann_filename)

#### load image filepaths

In [ ]:
# image_list = "/home/rte/data/paths/all_converted_jpg_images_shuf.txt"
# image_list = "/home/rte/data/paths/all_images_ordered.txt"
image_list = "/home/rte/data/paths/all_images_shuf.txt"

# image_folder = "/home/rte/data/images/web/120k/"
image_folder = "/mnt/hd2/images/all/"

In [ ]:
filepaths = []

with open(image_list, "r") as f:
    lines = f.readlines()
    print("length:",len(lines))
    print(lines[0])
for l in lines:
    # substrings = l.rsplit(",", 1)
    filepaths.append(l.strip())
    # image_ids.append(substrings[1].strip())

In [ ]:
# get the image paths for all images that we have features for
images = []

for i, f in enumerate(filepaths[:]):
    images.append(image_folder + f)
print(len(images))
print(images[:10])

In [ ]:
# resize to make sure we don't try and read more images than we have
filepaths = filepaths[:dataset_size]
print(len(filepaths))

In [ ]:
X = [] # images as arrays
dim = 256 # size of one dimension
f = dim * dim # size of vector
num_workers = 8

In [ ]:
def load_img(path, dim):
    im = Image.open(path).convert('RGB')
    img_x, img_y = im.size

    # im.show()

    if False:
        x_bigger = True if img_x > img_y else False
        ratio = img_x / img_y

        if x_bigger:
            factor = float(dim) / img_y 
        else:
            factor = float(dim) / img_x
        new_x = int(img_x * factor)
        new_y = int(img_y * factor)

        resized = im.resize((new_x, new_y), Image.ANTIALIAS)
        # resized.show()

        left = (new_x - dim)/2
        top = (new_y - dim)/2
        right = (new_x + dim)/2
        bottom = (new_y + dim)/2

        cropped = resized.crop((left, top, right, bottom))
#         cropped.show()

#     print("factor", factor)
#     print("ratio", ratio)
#     print(x_bigger)
#     print("img_x", img_x)
#     print("img_y", img_y)
#     print(new_x, new_y)
#     print("cropped", cropped.size)

    if True:
        # just resize to fill the box
        just_resize = im.resize((dim, dim), Image.ANTIALIAS)
    #     just_resize.show()
        return just_resize

    if False:
        if x_bigger:
            factor = float(dim) / img_x
        else:
            factor = float(dim) / img_y
        new_x = int(img_x * factor)
        new_y = int(img_y * factor)
    #     print("factor", factor)
    #     print("img_x", img_x)
    #     print("img_y", img_y)
        crop_resized = im.resize((new_x, new_y), Image.ANTIALIAS)

        # pad
        padded = Image.new("RGB", (dim, dim))
        padded.paste(crop_resized, ((dim - new_x)//2, (dim - new_y)//2))
        padded.show()
    
#     return just_resize
    return cropped

In [ ]:
def image_worker(filename):
    img_data = np.array(load_img(filename, dim))# - mean
    return img_data

def image_worker_flat(filename):
    img_data = np.array(load_img(filename, dim)).flatten()# - mean
    return img_data

def image_worker_vgg(filename):
    img, x = load_image(filename);
    return x

In [ ]:
mean = np.array([224.03429894, 223.6761458, 223.62304944]) # mean for 120k

In [ ]:
load_img(images[8], dim)

### Load images

Whole dataset

In [ ]:
def load_all_data():
    image_batch = images[:num_images]
    overall_start = time.time()
    with Pool(num_workers) as p:
        X = p.map(image_worker_flat, image_batch)
    print(f"loaded {len(X)} images | time taken: {time.time() - overall_start}")
    return X

In [ ]:
# X = load_all_data()

### Prepare feature extractor

In [ ]:
if mode == "ternary":
    model_path = "/home/rte/re-imaging/classification/checkpoints/ternary_20190911_9748x/diagram-sensor-unsure_vgg16-2000.hdf5"
    model = keras.models.load_model(model_path)
    model.summary()

In [ ]:
if mode == "vgg":
    model = keras.applications.VGG16(weights='imagenet', include_top=True)
    model.summary()

In [ ]:
if mode in ("vgg", "ternary"):
    def load_image(path):
        img = image.load_img(path, target_size=model.input_shape[1:3])
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        return img, x

In [ ]:
if mode in ("vgg", "ternary"):
    img, x = load_image(images[8])
    print("shape of x: ", x.shape)
    print("data type: ", x.dtype)
    plt.imshow(img)

In [ ]:
if mode == "vgg":
    predictions = model.predict(x)

    for _, pred, prob in decode_predictions(predictions)[0]:
        print("predicted %s with probability %0.3f" % (pred, prob))

In [ ]:
if mode == "vgg":
    feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
    feat_extractor.summary()
if mode == "ternary":
    feat_extractor = Model(inputs=model.input, outputs=model.get_layer("dense_1").output)
    feat_extractor.summary()

## Dimensionality Reduction

In [ ]:
n_components = 300

### IncrementalPCA

In [ ]:
ipca = IncrementalPCA(n_components=n_components, batch_size=16)

#### feed ipca in batches


In [ ]:
# old_batch_size = batch_size
# batch_size = 500
assert dataset_size % batch_size == 0, "dataset_size not a multiple of batch_size"
divisions = [x for x in range(0, dataset_size, batch_size)]
print("divisions:",divisions)
overall_start = time.time()

for div in divisions:
    X = []
#     print("batch:",div,div+batch_size)
#     for i, image in enumerate(images[div:div+batch_size]):
#         img_data = np.array(load_img(images[i], dim)) - mean
#         X.append(img_data.flatten())
    
    if mode in ("vgg", "ternary"):
        print("loading images for VGG or ternary classifier")
        start = time.time()
        with Pool(num_workers) as p:
            X = p.map(image_worker_vgg, images[div:div+batch_size])
        print(f"batch{div,div+batch_size} - time to load images: {time.time() - start}")    

        # extract features
        features = []
        for x in X:
            feat = feat_extractor.predict(x)[0]
            features.append(feat)
        print(f"extracted features of {len(X)} images, time taken: {time.time() - start}")
        X = features
    else:
        print("loading images for raw processing")
        start = time.time()
        with Pool(num_workers) as p:
            X = p.map(image_worker_flat, images[div:div+batch_size])
        print(f"batch{div,div+batch_size} - time to load images: {time.time() - start}")    
        
    print(X[0])
    print("size of data", len(X))
    start = time.time()
    ipca.partial_fit(X)
    print(f"partial fit time taken: {time.time() - start}")
print(f"total time taken: {time.time() - overall_start}")
# batch_size = old_batch_size

In [ ]:
# X_ipca = ipca.transform(X)

In [ ]:
# joblib_path = "ipca_2k_test.joblib"
# joblib_path = "ipca_120k_test.joblib"
joblib_path = ann_filename.split(".")[0] + "_ipca.joblib"

In [ ]:
print(joblib_path)

In [ ]:
# write joblib file to save ipca
with open(joblib_path, "wb") as f:
    joblib.dump(ipca, f)

In [ ]:
print("n_samples_seen_:", ipca.n_samples_seen_)

#### Load ipca
Testing only

In [ ]:
# with open(joblib_path, "rb") as f:
#     load_ipca = joblib.load(f) 

In [ ]:
# print(load_ipca.components_)

In [ ]:
# print(load_ipca.explained_variance_)

In [ ]:
# print(load_ipca.explained_variance_ratio_)

In [ ]:
# plt.plot(np.cumsum(load_ipca.explained_variance_ratio_))
# plt.title(joblib_path)
# plt.xlabel('number of components')
# plt.ylabel('cumulative explained variance');
# plot_savename = joblib_path.split(".")[0] + "_expvarrat.png"
# plt.savefig(plot_savename, dpi=300)

### Approximate Nearest Neighbours

In [ ]:
# pca
t = AnnoyIndex(300, 'angular')  # Length of item vector that will be indexed

# raw
# t = AnnoyIndex(256*256*3, 'angular')  # Length of item vector that will be indexed

t.set_seed(10)

In [ ]:
# feed annoy all in one go

# annoy_data = X_pca
# annoy_data = X_ipca
# # annoy_data = X
# print("length of annoy_data:", len(annoy_data))

# for i, item in enumerate(annoy_data):
#     t.add_item(i, item)

# start = time.time()
# t.build(10) # 10 trees
# t.save(ann_filename)
# print(f"added all to annoy, time taken: {time.time() - start}")

In [ ]:
# feed annoy in batches
assert dataset_size % batch_size == 0, "dataset_size not a multiple of batch_size"
divisions = [x for x in range(0, dataset_size, batch_size)]
print("divisions:",divisions)
overall_start = time.time()

counter = 0

for div in divisions:
    X = []
#     print("batch:",div,div+batch_size)
#     for i, image in enumerate(images[div:div+batch_size]):
#         img_data = np.array(load_img(images[i], dim)) - mean
#         X.append(img_data.flatten())

    if mode in ("vgg", "ternary"):
        print("loading images for VGG")
        start = time.time()
        with Pool(num_workers) as p:
            X = p.map(image_worker_vgg, images[div:div+batch_size])
        print(f"batch{div,div+batch_size} - time to load images: {time.time() - start}")    

        # extract features
        features = []
        for x in X:
            feat = feat_extractor.predict(x)[0]
            features.append(feat)
        print(f"extracted features of {len(X)} images, time taken: {time.time() - start}")
        X = features
    else:
        print("loading images for raw")
        start = time.time()
        with Pool(num_workers) as p:
            X = p.map(image_worker_flat, images[div:div+batch_size])
        print(f"batch{div,div+batch_size} - loading time taken: {time.time() - start}")
    start = time.time()
    ipca_X = ipca.transform(X)
    print(f"transform time taken: {time.time() - start}")
    start = time.time()
    for i, item in enumerate(ipca_X):
        t.add_item(counter, item)
        counter += 1
    print(f"annoy add items time taken: {time.time() - start}")
print(f"total time taken: {time.time() - overall_start}")

t.build(10) # 10 trees
t.save(ann_filename)

### Generate images to check ANN

run in loop for first 10 images

In [ ]:
num_n = 50

for i in range(10):
    target_index = i
    print("index", target_index)
    indexes = t.get_nns_by_item(target_index, num_n)
    print(indexes)
    target_images = np.array(images)[np.array(indexes)]
    
    xdim = 8
    ydim = 7

    fig, ax = plt.subplots(ydim, xdim)
    fig.set_size_inches(10, 12)
#     fig.patch.set_facecolor('0.98')
    #     print(filepaths[index].split(".")[0])
    title_string = f'annoy test: {mode}'
#     print(title_string)
    fig.suptitle(title_string, y=0.9)

    # display the query image
    query_image = mpimg.imread(images[target_index])
    ax[0, 0].imshow(query_image, cmap='Greys_r')
    ax[0, 0].set_title("query ID: " + filepaths[target_index].split(".")[0], pad=20)

    for i in range(xdim):
        ax[0, i]. axis('off')

    for y in range(1, ydim):
        for x in range(xdim):
#             print(f"x: {x}, y: {y}")
            grid_index = (y * xdim) + x - xdim
#             print(f"grid_index: {grid_index}")
            current_image = mpimg.imread(target_images[grid_index])
            ax[y, x].imshow(current_image, cmap='Greys_r')
            ax[y, x].axis('off')

#     plt.tight_layout(pad=0.5)
    savename = f"{ann_filename.split('.')[0]}_{target_index}.jpg"
    # savename = f'{query_image_idx:04}' + "_" + filepaths[index].split(".")[0] + "_strip_1m.jpg"
    plt.savefig(savename, dpi=300, bbox_inches='tight')
    # print("completed", index)
    # print("time taken", "{:.4f}".format(time.time() - start))